Last week, we anonomized Shailja's dataset that was pulled directly from SQL.  Engineering hasn't installed pyodbc on your systems, so a direct SQL pull still isn't going to happen, but we have 2902 anonomyzed rows to play with.

Our goals this week are to go over: 
pandas file i/o, vs load, vs pickle
row functions
looping over the rows of a dataframe
the apply function

Plotting if we have time.

In [ ]:
import pandas as pd
datecols = ['new_eta', 'original_eta', 'actual_receive_date']
datapath = '../status_analysis/data/'
df = pd.read_csv(datapath + 'anon.csv', parse_dates=datecols)

print df.head()

That's the only way we've been loading files: csv in pandas.  We could have loaded this file as text with the following code

In [ ]:
datapath = '../status_analysis/data/'

with open(datapath + 'anon.csv', 'r') as inf:
    file_text = inf.readlines()
    
for line in file_text[:10]:
    print line,

This is the underlying text in the comma separated values file, which you can verify by opening anon.csv with notebook rather than Excel.  Focusing on the open command:  we put the open(filename, mode) command in a with block so that the resource is closed once we finish the loop where we need that resource.  This is good practice in general, especially with SQL resources.

We can also use open in cojunction with one of python's best libraries, pickle, to store any python object in a binary (fast but human unreadable) file.

In [ ]:
import pickle
datapath = '../status_analysis/data/'
pickle.dump(df, open(datapath + 'binary.pkl', 'wb'))
loaded_df = pickle.load(open(datapath + 'binary.pkl', 'rb'))
len(loaded_df)

We can store multiple variables in a single pickle file

In [ ]:
import pickle
datapath = '../status_analysis/data/'

a_list = [1,2,3]
a_tuple = ['a', 6, []]
a_dict = {'a':1, 'b':2, 'c':3}

pickle.dump((a_list, a_tuple, a_dict), open(datapath + 'multiout.pkl', 'wb'))

b_list, b_tuple, b_dict = pickle.load(open(datapath + 'multiout.pkl', 'rb'))
print b_list, b_tuple, b_dict

Back to our dataframe: we can perform columnwise functions easily:

In [ ]:
print df.status_code.min()
print df.status_code.sum()

# This is how one performs a sumif
print df.loc[df.is_escalating == True].status_code.sum()

We can also perform a loop over every row and calculate something.  We shouldn't because it's slow, but the functionality exists.

In [ ]:
df['derived'] = -1

for i, row in df.iterrows():
    df.loc[i, 'derived'] = row['actual_receive_date'] - row['original_eta']
print df['derived'].max(), df['derived'].min()

Let's do a simple task: for all ids, see how many of them have more than one touchpoint.

In [ ]:
more_than_one = []

for id in df['id'].unique():
    sub = df[df['id'] == id]
    if len(sub) > 1:
        more_than_one.append(id)
print len(more_than_one)

We can do this elegantly with a pandas apply function

In [ ]:
def has_more_than_one(group):
    if len(group) > 1:
        return True
    return None

results = df.groupby('id').apply(has_more_than_one)
print results.dropna().head()
print type(results)

Note that this returned a series.  If you want more than one column returned from your apply funciton, it's pretty easy.

In [ ]:
def two_outputs(group):
    if len(group) > 1:
        return pd.Series({'multi_touch': True, 'n_contacts': len(group)})
    return pd.Series({'multi_touch': None, 'n_contacts': len(group)})

results = df.groupby('id').apply(two_outputs)
print results.dropna().head()
